In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
import cPickle as pickle
import html5lib
import os
import ipyparallel as ipp

from bs4 import BeautifulSoup as bs
from urllib2 import urlopen
from urllib2 import Request
from urllib2 import URLError

reload(sys)
sys.setdefaultencoding('utf-8')
sys.setrecursionlimit(50000)

%matplotlib inline

In [3]:
#Corremos el cluster
rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()

# Cargamos los datos de las propiedades

In [4]:
#Cargo toda la info
propertyD = pd.read_csv('../../../properati-AR-2017-08-01-properties-sell.csv')
propertyData = propertyD[(propertyD['price_aprox_usd'].isnull() == False)]

In [11]:
propertyData.info()
pickle.dump(propertyData,open("save.p","wb"))
#Todas tienen la url.

# Completamos los cuartos con las descripciones.

In [5]:
caracteristicas = propertyData[['id','description','title','properati_url','rooms']]
caracteristicas.head(10)

,id,description,title,properati_url,rooms
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,http://www.properati.com.ar/udh_venta_casa_mer...,NaN
1,a205c53378e7879cf7519659317fb410c060d20e,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,http://www.properati.com.ar/20hi_venta_casa_me...,NaN
4,9446052c949050359fb08b9f1ba83ba2eaa20c99,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,http://www.properati.com.ar/4m4m_venta_casa_ma...,5.0
5,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,http://www.properati.com.ar/518a_venta_departa...,2.0
6,535441f569473f6e5d687c38b27d9b9c6e69e284,"Venta - Local - Argentina, Buenos Aires, Zona ...",Local en Venta ubicado en San Isidro,http://www.properati.com.ar/52p4_venta_local_s...,1.0
8,27d87ac71e7c00afa9fa7428b943ec263ff68bb4,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Quilquihue,http://www.properati.com.ar/568j_venta_casa_sa...,4.0
9,df77a06585671e637d2d86f8d22b70d61cfecb18,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Rio Hermoso,http://www.properati.com.ar/568m_venta_casa_sa...,3.0
10,61f3789618edabeaa7b5fc06ff36112f8a06557f,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Pilar,http://www.properati.com.ar/59ec_venta_departa...,2.0
11,c7ac98ce42a2689a8aee59fb1e91c6b7194c033c,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Centro,http://www.properati.com.ar/5b9n_venta_casa_sa...,4.0
12,e842ed22eac89b9bffff146377abfd5bceb0b0cd,"Venta - Casa - Argentina, Buenos Aires, Zona N...",Casa en Venta ubicado en Benavidez,http://www.properati.com.ar/5paq_venta_casa_be...,6.0


In [8]:
#caracteristicas['rooms'].isnull().sum()
caracteristicas = caracteristicas[0:10000]

In [7]:
def chequearCuartos(x):
    if(not pd.isnull(x[4])):
        return x[4]
    return buscarCuartos(x[1]) or buscarCuartos(x[2])
        
def buscarCuartos(x):
    
    caract = ""
    caractViviendas = ['dorm','hab','cuarto'] 
    
    if( x == None or x == ""):
        return None
    
    if type( x ) is str:
        x = x.lower()
    if type( x ) is float:
        x = str(x).lower()
    #Queda una lista separada por comas.    
    x = x.split()
    
    for j in xrange(len(caractViviendas)):
        
        aux = ""
        palabraAct = caractViviendas[j]
        
        if( palabraAct in x ):
            indice = x.index( palabraAct )
            #caract = buscarCantidadDorm(x,indice)
            caract = x[indice-1]
            break;
            
        else:
            for ind in xrange( len(x) ):
                if( palabraAct in x[ind] ):
                    #caract = buscarCantidadDorm(x,ind)
                    caract = x[ind-1]
                    break;
    if(caract.isdigit()):
        return int(caract)
    return None  

In [9]:
caracteristicas['rooms'] = caracteristicas.apply(chequearCuartos, axis=1)

In [42]:
#caracteristicas['rooms'].isnull().sum()#62257

1

In [10]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,http://www.properati.com.ar/udh_venta_casa_mer...,NaN
1,a205c53378e7879cf7519659317fb410c060d20e,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,http://www.properati.com.ar/20hi_venta_casa_me...,3.0
4,9446052c949050359fb08b9f1ba83ba2eaa20c99,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,http://www.properati.com.ar/4m4m_venta_casa_ma...,5.0
5,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,http://www.properati.com.ar/518a_venta_departa...,2.0
6,535441f569473f6e5d687c38b27d9b9c6e69e284,"Venta - Local - Argentina, Buenos Aires, Zona ...",Local en Venta ubicado en San Isidro,http://www.properati.com.ar/52p4_venta_local_s...,1.0
8,27d87ac71e7c00afa9fa7428b943ec263ff68bb4,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Quilquihue,http://www.properati.com.ar/568j_venta_casa_sa...,4.0
9,df77a06585671e637d2d86f8d22b70d61cfecb18,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Rio Hermoso,http://www.properati.com.ar/568m_venta_casa_sa...,3.0
10,61f3789618edabeaa7b5fc06ff36112f8a06557f,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Pilar,http://www.properati.com.ar/59ec_venta_departa...,2.0
11,c7ac98ce42a2689a8aee59fb1e91c6b7194c033c,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Centro,http://www.properati.com.ar/5b9n_venta_casa_sa...,4.0
12,e842ed22eac89b9bffff146377abfd5bceb0b0cd,"Venta - Casa - Argentina, Buenos Aires, Zona N...",Casa en Venta ubicado en Benavidez,http://www.properati.com.ar/5paq_venta_casa_be...,6.0


# Ahora añadimos los ambientes

In [13]:
caracteristicas['ambientes'] = pd.Series(None, index=caracteristicas.index)

In [11]:
def chequearAmbientes(x):
    
    if(not pd.isnull(x[5])):
        return x[5]
    return buscarAmbientes(x[1]) or buscarAmbientes(x[2])
        
def buscarAmbientes(x):
    
    caract = ""
    caractViviendas = ['amb','monoamb'] 
    
    if( x == None or x == ""):
        return None
    
    if type( x ) is str:
        x = x.lower()
    if type( x ) is float:
        x = str(x).lower()
    #Queda una lista separada por comas.    
    x = x.split()
    
    for j in xrange(len(caractViviendas)):
        
        aux = ""
        palabraAct = caractViviendas[j]
        
        if( palabraAct in x ):
            indice = x.index( palabraAct )
            if(palabraAct == 'monoamb'):caract = '1'
            else: caract = x[indice-1]
            break;
            
        else:
            for ind in xrange( len(x) ):
                if( palabraAct in x[ind] ):
                    if(palabraAct == 'monoamb'):caract = '1'
                    else: caract = x[ind-1]
                    break;
    
    if(caract.isdigit()):
        return int(caract)
    return None  

In [14]:
caracteristicas['ambientes'] = caracteristicas.apply(chequearAmbientes, axis=1)

In [15]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms,ambientes
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,http://www.properati.com.ar/udh_venta_casa_mer...,NaN,NaN
1,a205c53378e7879cf7519659317fb410c060d20e,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,http://www.properati.com.ar/20hi_venta_casa_me...,3.0,4.0
4,9446052c949050359fb08b9f1ba83ba2eaa20c99,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,http://www.properati.com.ar/4m4m_venta_casa_ma...,5.0,3.0
5,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,http://www.properati.com.ar/518a_venta_departa...,2.0,2.0
6,535441f569473f6e5d687c38b27d9b9c6e69e284,"Venta - Local - Argentina, Buenos Aires, Zona ...",Local en Venta ubicado en San Isidro,http://www.properati.com.ar/52p4_venta_local_s...,1.0,NaN
8,27d87ac71e7c00afa9fa7428b943ec263ff68bb4,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Quilquihue,http://www.properati.com.ar/568j_venta_casa_sa...,4.0,NaN
9,df77a06585671e637d2d86f8d22b70d61cfecb18,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Rio Hermoso,http://www.properati.com.ar/568m_venta_casa_sa...,3.0,NaN
10,61f3789618edabeaa7b5fc06ff36112f8a06557f,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Pilar,http://www.properati.com.ar/59ec_venta_departa...,2.0,NaN
11,c7ac98ce42a2689a8aee59fb1e91c6b7194c033c,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Centro,http://www.properati.com.ar/5b9n_venta_casa_sa...,4.0,NaN
12,e842ed22eac89b9bffff146377abfd5bceb0b0cd,"Venta - Casa - Argentina, Buenos Aires, Zona N...",Casa en Venta ubicado en Benavidez,http://www.properati.com.ar/5paq_venta_casa_be...,6.0,NaN


# Por ultimo vamos a buscar en internet cuando no haya ninguno de los 2 datos.

In [16]:
caracteristicas['webp'] = pd.Series(None, index=caracteristicas.index)

In [29]:
def chequearURL(x):
    
    if( pd.isnull(x[3]) ): return None
    if( not pd.isnull(x[4]) or not pd.isnull(x[5]) ): return None
    
    return buscarURL(x[3])
        
def buscarURL(x):
    
    req = Request(x)
    try:
        resp = urlopen(req, timeout=10)
    except URLError:
        print "Bad URL or timeout"
        return None

    resp_page = resp.read()
    
    return resp_page

In [30]:
caracteristicas['webp'] = caracteristicas.apply(chequearURL, axis=1)

In [31]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms,ambientes,webp
0,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,CASA CON LOCAL EN VENTA EN PADUA.2 DORMITORIOS...,CASA EN VENTA,http://www.properati.com.ar/udh_venta_casa_mer...,NaN,NaN,None
1,a205c53378e7879cf7519659317fb410c060d20e,CHALET VENTA 4 AMBIENTES PADUA.EXCELENTE CHALE...,CASA EN VENTA,http://www.properati.com.ar/20hi_venta_casa_me...,3.0,4.0,None
4,9446052c949050359fb08b9f1ba83ba2eaa20c99,Casa Multifamiliar en Mataderos!!! Sobre lote ...,CASA EN VENTA,http://www.properati.com.ar/4m4m_venta_casa_ma...,5.0,3.0,None
5,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Arcos de Pilar,http://www.properati.com.ar/518a_venta_departa...,2.0,2.0,None
6,535441f569473f6e5d687c38b27d9b9c6e69e284,"Venta - Local - Argentina, Buenos Aires, Zona ...",Local en Venta ubicado en San Isidro,http://www.properati.com.ar/52p4_venta_local_s...,1.0,NaN,None
8,27d87ac71e7c00afa9fa7428b943ec263ff68bb4,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Quilquihue,http://www.properati.com.ar/568j_venta_casa_sa...,4.0,NaN,None
9,df77a06585671e637d2d86f8d22b70d61cfecb18,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Rio Hermoso,http://www.properati.com.ar/568m_venta_casa_sa...,3.0,NaN,None
10,61f3789618edabeaa7b5fc06ff36112f8a06557f,"Venta - Departamento - Argentina, Buenos Aires...",Departamento en Venta ubicado en Pilar,http://www.properati.com.ar/59ec_venta_departa...,2.0,NaN,None
11,c7ac98ce42a2689a8aee59fb1e91c6b7194c033c,"Venta - Casa - Argentina, Patagonia, San Marti...",Casa en Venta ubicado en Centro,http://www.properati.com.ar/5b9n_venta_casa_sa...,4.0,NaN,None
12,e842ed22eac89b9bffff146377abfd5bceb0b0cd,"Venta - Casa - Argentina, Buenos Aires, Zona N...",Casa en Venta ubicado en Benavidez,http://www.properati.com.ar/5paq_venta_casa_be...,6.0,NaN,None


In [32]:
aux = caracteristicas[ pd.isnull(caracteristicas['webp']) == True ]
aux.count()

id               10000
description       9999
title            10000
properati_url    10000
rooms             5703
ambientes         4434
webp                 0
dtype: int64

In [28]:
pickle.dump(caracteristicas,open("save1.p","wb"))

# Rellenamos los datos que nos interezan

In [92]:
def chequearCuartosD(x):
    if(not pd.isnull(x[4])): return x[4]
    if(not pd.isnull(x[6])): return buscarCuartosD(x[6])
    return None

def buscarCuartosD(x):
    index = x.find('Dorm')
    
    cant = x[index+11:index+16]
    
    total = ''
    for x in cant:
        if(x.isdigit()):
            total+=x
    if(total == ''): return None
    return int(total)

In [93]:
caracteristicas['rooms'] = caracteristicas.apply(chequearCuartosD, axis=1)

In [94]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms,ambientes,adc
12752,b353a4eb781fcae6e091eb596d466c32db2e5d1f,CODIGO: 1974-283 ubicado en: Tucuman 1600 - P...,Hermoso Dpto de categoria de 4 Ambientes de 90...,http://www.properati.com.ar/q490_venta_departa...,NaN,4.0,None
12753,301f38cd7af8cbca6ab8b484e4fb6c6389af6d57,CODIGO: 2912-C1013V ubicado en: 25 de mayo 220...,Lanus - Casa 4 ambientes amplios con patio y d...,http://www.properati.com.ar/q534_venta_casa_la...,NaN,4.0,None
12754,b783b4bec9db78c790cf046cb87cf22570f3e9e0,CODIGO: 1019-V171 ubicado en: PATRON 5460 - P...,"4 AMBIENTES DE PRIMERA CATEGORIA , DOS COCHERA...",http://www.properati.com.ar/qqss_venta_departa...,NaN,4.0,None
12755,10c126be2a900ba25d1147e693093b9fabd30a8a,Departamento de tres ambientes interno totalme...,"Tres ambientes, contrafrente. reciclado. COn e...",http://www.properati.com.ar/qsxk_venta_departa...,3.0,NaN,None
12756,cba8f065e491267ea95940203e51d723f3a6501b,CODIGO: 217-E-139 ubicado en: Dolores 100 - P...,CHALET VILLA SARMIENTO,http://www.properati.com.ar/qufr_venta_casa_vi...,1.0,NaN,<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...
12757,14c5e6bafe94b86ba7d34dd8b7eac2cad61471a4,Excelente Departamento Céntrico de estilo mode...,9851 - Patagonia I,http://www.properati.com.ar/qwka_venta_departa...,2.0,NaN,None
12758,e1789c13288d37fbfae49227e62573c62f7cacbc,"En inmejorable zona de la ciudad, se ofrecen e...",DEPTO 2 AMBIENTES DE CATEGORIA A ESTRENAR - V...,http://www.properati.com.ar/qwlg_venta_departa...,2.0,2.0,None
12759,8d2c19daddad8d3e392f467a44cd9c5d0c8dd700,DEPARTAMENTO 3 AMBIENTES CON COCHERA Y BAULERA...,DEPTO 3 AMB COCHERA Y BAULERA A ESTRENAR- APTO...,http://www.properati.com.ar/qwlh_venta_departa...,3.0,3.0,None
12760,5763f9177448922af81c7442e39453cb510ad50c,MUY BUEN DEPARTAMENTO DE 3 AMBIENTES A LA CALL...,DEPARTAMENTO 3 AMBIENTES EN ZONA EX TERMINAL,http://www.properati.com.ar/qwpn_venta_departa...,3.0,3.0,None
12761,9773220b1b72ca98bdcccbbd580047d56080203c,Vendo hermosa propiedad en zona centro de Ober...,Casa con Local en Venta. Ambientes. Dormitorio...,http://www.properati.com.ar/qwsb_venta_casa_obera,9.0,NaN,None


In [100]:
def chequearAmbientesD(x):
    if(not pd.isnull(x[5])): return x[5]
    if(not pd.isnull(x[6])): return buscarAmbientesD(x[6])
    return None

def buscarAmbientesD(x):
    index = x.find('Ambientes')
    
    cant = x[index+9:index+14]
    
    total = ''
    for x in cant:
        if(x.isdigit()):
            total+=x
    if(total == ''): return None
    return int(total)

In [101]:
caracteristicas['ambientes'] = caracteristicas.apply(chequearAmbientesD, axis=1)

In [102]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms,ambientes,adc,baños
12752,b353a4eb781fcae6e091eb596d466c32db2e5d1f,CODIGO: 1974-283 ubicado en: Tucuman 1600 - P...,Hermoso Dpto de categoria de 4 Ambientes de 90...,http://www.properati.com.ar/q490_venta_departa...,NaN,4.0,None,NaN
12753,301f38cd7af8cbca6ab8b484e4fb6c6389af6d57,CODIGO: 2912-C1013V ubicado en: 25 de mayo 220...,Lanus - Casa 4 ambientes amplios con patio y d...,http://www.properati.com.ar/q534_venta_casa_la...,NaN,4.0,None,NaN
12754,b783b4bec9db78c790cf046cb87cf22570f3e9e0,CODIGO: 1019-V171 ubicado en: PATRON 5460 - P...,"4 AMBIENTES DE PRIMERA CATEGORIA , DOS COCHERA...",http://www.properati.com.ar/qqss_venta_departa...,NaN,4.0,None,NaN
12755,10c126be2a900ba25d1147e693093b9fabd30a8a,Departamento de tres ambientes interno totalme...,"Tres ambientes, contrafrente. reciclado. COn e...",http://www.properati.com.ar/qsxk_venta_departa...,3.0,NaN,None,NaN
12756,cba8f065e491267ea95940203e51d723f3a6501b,CODIGO: 217-E-139 ubicado en: Dolores 100 - P...,CHALET VILLA SARMIENTO,http://www.properati.com.ar/qufr_venta_casa_vi...,1.0,2.0,<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...,NaN
12757,14c5e6bafe94b86ba7d34dd8b7eac2cad61471a4,Excelente Departamento Céntrico de estilo mode...,9851 - Patagonia I,http://www.properati.com.ar/qwka_venta_departa...,2.0,NaN,None,NaN
12758,e1789c13288d37fbfae49227e62573c62f7cacbc,"En inmejorable zona de la ciudad, se ofrecen e...",DEPTO 2 AMBIENTES DE CATEGORIA A ESTRENAR - V...,http://www.properati.com.ar/qwlg_venta_departa...,2.0,2.0,None,NaN
12759,8d2c19daddad8d3e392f467a44cd9c5d0c8dd700,DEPARTAMENTO 3 AMBIENTES CON COCHERA Y BAULERA...,DEPTO 3 AMB COCHERA Y BAULERA A ESTRENAR- APTO...,http://www.properati.com.ar/qwlh_venta_departa...,3.0,3.0,None,NaN
12760,5763f9177448922af81c7442e39453cb510ad50c,MUY BUEN DEPARTAMENTO DE 3 AMBIENTES A LA CALL...,DEPARTAMENTO 3 AMBIENTES EN ZONA EX TERMINAL,http://www.properati.com.ar/qwpn_venta_departa...,3.0,3.0,None,NaN
12761,9773220b1b72ca98bdcccbbd580047d56080203c,Vendo hermosa propiedad en zona centro de Ober...,Casa con Local en Venta. Ambientes. Dormitorio...,http://www.properati.com.ar/qwsb_venta_casa_obera,9.0,NaN,None,NaN


# Buscamos Baños

In [103]:
caracteristicas['Baños'] = pd.Series(None, index=caracteristicas.index)

In [105]:
def chequearBaniosD(x):
    if(not pd.isnull(x[6])): return buscarBaniosD(x[6])
    return None

def buscarBaniosD(x):
    index = x.find('Baño')
    
    cant = x[index+4:index+9]
    
    total = ''
    for x in cant:
        if(x.isdigit()):
            total+=x
    if(total == ''): return None
    return int(total)

In [109]:
caracteristicas['banios'] = caracteristicas.apply(chequearBaniosD, axis=1)

In [110]:
caracteristicas.head(10)

,id,description,title,properati_url,rooms,ambientes,adc,baños,Baños,banios
12752,b353a4eb781fcae6e091eb596d466c32db2e5d1f,CODIGO: 1974-283 ubicado en: Tucuman 1600 - P...,Hermoso Dpto de categoria de 4 Ambientes de 90...,http://www.properati.com.ar/q490_venta_departa...,NaN,4.0,None,None,NaN,None
12753,301f38cd7af8cbca6ab8b484e4fb6c6389af6d57,CODIGO: 2912-C1013V ubicado en: 25 de mayo 220...,Lanus - Casa 4 ambientes amplios con patio y d...,http://www.properati.com.ar/q534_venta_casa_la...,NaN,4.0,None,None,NaN,None
12754,b783b4bec9db78c790cf046cb87cf22570f3e9e0,CODIGO: 1019-V171 ubicado en: PATRON 5460 - P...,"4 AMBIENTES DE PRIMERA CATEGORIA , DOS COCHERA...",http://www.properati.com.ar/qqss_venta_departa...,NaN,4.0,None,None,NaN,None
12755,10c126be2a900ba25d1147e693093b9fabd30a8a,Departamento de tres ambientes interno totalme...,"Tres ambientes, contrafrente. reciclado. COn e...",http://www.properati.com.ar/qsxk_venta_departa...,3.0,NaN,None,None,NaN,None
12756,cba8f065e491267ea95940203e51d723f3a6501b,CODIGO: 217-E-139 ubicado en: Dolores 100 - P...,CHALET VILLA SARMIENTO,http://www.properati.com.ar/qufr_venta_casa_vi...,1.0,2.0,<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html clas...,None,NaN,None
12757,14c5e6bafe94b86ba7d34dd8b7eac2cad61471a4,Excelente Departamento Céntrico de estilo mode...,9851 - Patagonia I,http://www.properati.com.ar/qwka_venta_departa...,2.0,NaN,None,None,NaN,None
12758,e1789c13288d37fbfae49227e62573c62f7cacbc,"En inmejorable zona de la ciudad, se ofrecen e...",DEPTO 2 AMBIENTES DE CATEGORIA A ESTRENAR - V...,http://www.properati.com.ar/qwlg_venta_departa...,2.0,2.0,None,None,NaN,None
12759,8d2c19daddad8d3e392f467a44cd9c5d0c8dd700,DEPARTAMENTO 3 AMBIENTES CON COCHERA Y BAULERA...,DEPTO 3 AMB COCHERA Y BAULERA A ESTRENAR- APTO...,http://www.properati.com.ar/qwlh_venta_departa...,3.0,3.0,None,None,NaN,None
12760,5763f9177448922af81c7442e39453cb510ad50c,MUY BUEN DEPARTAMENTO DE 3 AMBIENTES A LA CALL...,DEPARTAMENTO 3 AMBIENTES EN ZONA EX TERMINAL,http://www.properati.com.ar/qwpn_venta_departa...,3.0,3.0,None,None,NaN,None
12761,9773220b1b72ca98bdcccbbd580047d56080203c,Vendo hermosa propiedad en zona centro de Ober...,Casa con Local en Venta. Ambientes. Dormitorio...,http://www.properati.com.ar/qwsb_venta_casa_obera,9.0,NaN,None,None,NaN,None


In [113]:
aux = caracteristicas[[ 'id','rooms','ambientes','banios' ] ]

In [114]:
pickle.dump(aux,open("save2.p","wb"))

In [116]:
pickle.dump(caracteristicas,open("aux.p","wb"))